<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 3<br/>*Depedency parser* pour le français dans spaCy

**Objectif**

Évaluer l'analyseur syntaxique en dépendances fourni par spaCy dans le modèle `fr_core_news_sm`, puis le comparer avec un analyseur entraîné par vous-mêmes.  Les données sont les mêmes qu'au Labo 2 et la démarche du labo est similaire aussi.

## 1. Prise en main de l'analyseur de spaCy

In [1]:
import spacy
nlp = spacy.load("fr_core_news_sm") # charge la pipeline

**1a.** Pour la pipeline `fr_core_news_sm`, veuillez afficher les traitements disponibles, puis désactiver tous les traitements sauf `tok2vec`, `morphologizer` et `parser`, puis vérifiez que la désactivation a bien fonctionné.

In [2]:
# Veuillez écrire votre code ici.
print("Traitements disponibles :", nlp.pipe_names)
for pipe in nlp.pipe_names:
    if pipe not in {"tok2vec", "morphologizer","parser"}:
        nlp.disable_pipe(pipe)

print("Traitements activés :", nlp.pipe_names)

Traitements disponibles : ['tok2vec', 'morphologizer', 'parser', 'attribute_ruler', 'lemmatizer', 'ner']
Traitements activés : ['tok2vec', 'morphologizer', 'parser']


In [3]:
from spacy.lang.fr.examples import sentences
s1 = sentences[2] # prenons la 3e phrase comme exemple

**1b.** Veuillez analyser `s1` avec la pipeline `nlp` puis afficher chaque token, son POS tag, et son étiquette indiquant la relation de dépendance (entre crochets, après le token).  Quelle information essentielle manque dans cette représentation ?

Note : le *morphologizer* fournit aussi les POS tags.  La liste des tags possibles est [fournie par spaCy](https://spacy.io/models/fr#fr_core_news_md-labels).  

In [4]:
# Veuillez écrire votre code et votre réponse ici.
doc = nlp(s1)
print(doc.text)
for token in doc:
    print(f"{token.text} [{token.pos_}, {token.dep_}]")

# Je ne suis pas sûr mais il manque la relation entre San et Francisco ou bien encore robots et coursiers

San Francisco envisage d'interdire les robots coursiers sur les trottoirs
San [PRON, nsubj]
Francisco [PROPN, flat:name]
envisage [VERB, ROOT]
d' [ADP, case]
interdire [NOUN, obl:arg]
les [DET, det]
robots [NOUN, obj]
coursiers [ADJ, amod]
sur [ADP, case]
les [DET, det]
trottoirs [NOUN, nmod]


**1c.** Veuillez afficher tous les groupes de mots qui sont soit des `nsubj` soit des `obj` dans la phrase `s1` (c'est à dire les sujets et les objets du verbe).   Indication : le sous-arbre d'un token *t* est accessible comme `t.subtree`. 

In [5]:
# Veuillez écrire votre code ici.
for token in doc:
    if token.dep_ in ["nsubj", "obj"]:
        subtree_words = " ".join([t.text for t in token.subtree])
        print(f"{token.dep_}: {subtree_words}")

nsubj: San Francisco
obj: les robots coursiers sur les trottoirs


## 2. Évaluation quantitative de l'analyseur sur une phrase 

Les données sont les mêmes que celles du Labo 2.  Vous les avez déjà transformées au Labo 2 dans un format utilisable par spaCy, dans un dossier nommé `Labo2/spacy_data` que vous allez réutiliser.  Les trois fichiers contiennent des phrases en français annotées aussi avec les arbres de dépendance.  Le fichier `fr-ud-train.conllu` est destiné à l'entraînement, `fr-ud-dev.conllu` au réglage des paramètres, et `fr-ud-test.conllu` à l'évaluation finale.

**2a.** En inspectant un des fichiers d'origine avec un éditeur texte, veuillez indiquer dans quelles colonnes se trouvent les informations sur les relations de dépendance, et comment elles sont représentées.

In [6]:
# Veuillez écrire votre réponse dans cette cellule.
# Sur la huitième colonne, donc la colonne 7 en informatique vu que le départ est à 0

In [7]:
from spacy.tokens import DocBin, Doc
test_data = DocBin().from_disk("spacy_data/fr-ud-test.spacy")
# for doc in test_data.get_docs(nlp.vocab):  # exemple
#     for sent in doc.sents:
#         print(sent)

**2b**. On rapplle que les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Ici, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la *7e phrase du 2e document des données de test* dans une variable nommée `s2`.
* Veuillez afficher cette phrase (elle commence par "Trois ans").

In [8]:
# Veuillez écrire votre code ici.
s2 = list(test_data.get_docs(nlp.vocab))
s2 = list(s2[1].sents)[6]

print(s2)
print(type(s2))

Trois ans plus tard, il tient un discours sur la crise.
<class 'spacy.tokens.span.Span'>


**2c.** En utilisant `displaCy` comme expliqué [ici](https://spacy.io/usage/visualizers) veuillez afficher graphiquement l'arbre de dépendances de la phrase `s2` tel qu'il est fourni dans les données.  Pour être affichée, la phrase doit être transformée en objet `Doc`.

In [9]:
from spacy import displacy


In [10]:
# Veuillez écrire votre code ici.
from IPython.display import HTML, display

display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(s2, style="dep", jupyter=False)
)))

**2d.** En utilisant `displaCy`, veuillez également afficher l'arbre de dépendances calculé par la pipeline `nlp` pour cette même phrase `s2`.  Pour être analysée et affichée, la phrase doit être transformée en objet `Doc`.

In [11]:
# Veuillez écrire votre code ici.
doc2 = nlp(s2.as_doc())
display(HTML('<span class="tex2jax_ignore">{}</span>'.format(
    displacy.render(doc2, style="dep", jupyter=False)
)))

**2e.** Veuillez comparer les deux arbres de dépendances et indiquer ici les différences.  Quel est le taux de correction de la pipeline `nlp` sur cette phrase ?

Suggestion : il peut être utile de sauvegarder les deux arbres dans des images SVG, en écrivant dans un fichier le résultat retourné par `displacy.render` avec l'option `jupyter = False`.

In [12]:
# Veuillez écrire votre réponse ici.
with open("original_tree.svg", "w", encoding="utf-8") as f:
    f.write(displacy.render(s2, style="dep", jupyter=False))
    
with open("predicted_tree.svg", "w", encoding="utf-8") as f:
    f.write(displacy.render(doc2, style="dep", jupyter=False))

# On peut voir que la différence se trouve dans le groupe "ans plus tard il tient"
# L'arbre original TODO !!!!!!!!!!!


original_tokens = [(token.text, token.dep_, token.head.text) for token in s2]
predicted_tokens = [(token.text, token.dep_, token.head.text) for token in doc2]

# Calculer le taux de correction
correct = sum(1 for g, p in zip(original_tokens, predicted_tokens) if g == p)
total = len(original_tokens)
accuracy = correct / total

print(f"Taux de correction : {accuracy:.2%}")

Taux de correction : 84.62%


**2f.**  Veuillez appliquer le `Scorer` de spaCy (voir Labo 2) et afficher les deux scores qu'il produit pour l'analyse en dépendances (avec trois décimales après la virgule).  Retrouvez-vous les scores de la question précédente ? Pourquoi ?

In [13]:
from spacy.scorer import Scorer
from spacy.training import Example

In [14]:
# Veuillez écrire votre code et votre réponse ici.
scorer = Scorer()

example = Example(s2.as_doc(), doc2)
scores = scorer.score([example])

print(f"UAS : {scores['dep_uas']:.3f}")
print(f"LAS : {scores['dep_las']:.3f}")

# Comparer avec le calcul manuel précédent
print(f"\nComparaison avec le calcul manuel précédent:")
print(f"Taux de correction calculé manuellement: {accuracy:.3f}")

#On trouve en effet un score différent,,, je sais pas pourquoi par contre TODO

UAS : 0.818
LAS : 0.818

Comparaison avec le calcul manuel précédent:
Taux de correction calculé manuellement: 0.846


## 3. Évaluation du *dependency parser* de `fr_core_news_sm` sur l'ensemble des phrases test

**3a.** Veuillez calculer les deux scores qui caractérisent l'analyseur en dépendances de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se comparent ces scores avec ceux mentionnés [dans la documentation de fr_core_news_sm](https://spacy.io/models/fr#fr_core_news_sm) ?

In [15]:
# Veuillez écrire votre code ici.
examples = []
# Parcourir tous les documents et phrases dans les données de test
for doc in test_data.get_docs(nlp.vocab):
    for sent in doc.sents:
        
        parsed_sent = nlp(sent.text)
        example = Example(sent.as_doc(), parsed_sent)
        examples.append(example)

scores = scorer.score(examples)

print(f"UAS : {scores['dep_uas']:.3f}")
print(f"LAS : {scores['dep_las']:.3f}")

print("\nComparaison avec les scores de la documentation:")
print("Documentation fr_core_news_sm: UAS = 88%, LAS = 84%")

#On voit qu'ils sont bien moins bon qu'advertised ... TODO

UAS : 0.760
LAS : 0.619

Comparaison avec les scores de la documentation:
Documentation fr_core_news_sm: UAS = 88%, LAS = 84%


**3b.** Le *scorer* fournit également des scores détaillés pour chaque type de relation de dépendances.  Veuillez afficher ces valeurs dans un tableau proprement formaté, trié par score F1 décroissant, avec trois décimales.

In [18]:
# Veuillez écrire votre code ici.
import pandas as pd

scores_per_type = scores["dep_las_per_type"]

df = pd.DataFrame.from_dict(scores_per_type, orient="index")
df.reset_index(inplace=True)
df.columns = ["Relation", "Precision", "Recall", "F1-Score"]

# Trier par F1 décroissant
df = df.sort_values(by="F1-Score", ascending=False)

# Arrondir les scores à trois décimales
df = df.round(3)

# Afficher le tableau
print(df.to_string(index=False))

    Relation  Precision  Recall  F1-Score
         det      0.770   0.928     0.842
        case      0.775   0.854     0.813
          cc      0.794   0.797     0.795
  nsubj:pass      0.860   0.683     0.761
        mark      0.779   0.723     0.750
    aux:pass      0.925   0.628     0.748
         cop      0.741   0.720     0.730
       nsubj      0.676   0.757     0.714
      nummod      0.664   0.750     0.705
        root      0.795   0.617     0.695
      advmod      0.700   0.678     0.689
         obj      0.694   0.684     0.689
        amod      0.743   0.588     0.656
   flat:name      0.632   0.610     0.621
        nmod      0.594   0.539     0.565
       xcomp      0.613   0.489     0.544
   acl:relcl      0.547   0.500     0.522
       fixed      0.337   0.523     0.410
         acl      0.416   0.403     0.409
        iobj      0.560   0.318     0.406
       advcl      0.382   0.402     0.392
       ccomp      0.317   0.465     0.377
        conj      0.370   0.381   

## 4. Entraîner puis évaluer un nouveau *parser* français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent (voir le Labo 2 et les [instructions de spaCy](https://spacy.io/usage/training#quickstart)).

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous gardez seulement les composants `morphologizer` et `parser` dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [20]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques (*à indiquer dans config.cfg*), pour estimer le temps nécessaire et observer les messages affichés.  Augmentez progressivement le nombre d'époques, jusqu'à ce que les scores sur le jeu de validation n'augmentent plus (si vous avez le temps).  Pendant combien d'époques entraînez-vous au final ?

In [22]:
# Note : il vaut mieux exécuter cela directement dans une fenêtre de commande, pour voir les logs en temps réel.
!python -m spacy train config.cfg \
  --output ./myDEPparser1 \
  --paths.train ./spacy_data/fr-ud-train.spacy \
  --paths.dev ./spacy_data/fr-ud-dev.spacy \
  --verbose

[2025-04-03 07:55:20,039] [DEBUG] Config overrides from CLI: ['paths.train', 'paths.dev']
ℹ Saving to output directory: myDEPparser1
ℹ Using CPU

=========================== Initializing pipeline ===========================
[2025-04-03 07:55:24,383] [INFO] Set up nlp object from config
[2025-04-03 07:55:24,399] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-04-03 07:55:24,401] [DEBUG] Loading corpus from path: spacy_data/fr-ud-train.spacy
[2025-04-03 07:55:24,402] [INFO] Pipeline: ['tok2vec', 'morphologizer', 'parser']
[2025-04-03 07:55:24,406] [INFO] Created vocabulary
[2025-04-03 07:55:24,406] [INFO] Finished initializing nlp object
[2025-04-03 07:56:19,076] [INFO] Initialized pipeline components: ['tok2vec', 'morphologizer', 'parser']
✔ Initialized pipeline

============================= Training pipeline =============================
[2025-04-03 07:56:19,114] [DEBUG] Loading corpus from path: spacy_data/fr-ud-dev.spacy
[2025-04-03 07:56:19,117] [DEBUG] Loading c

In [ ]:
# Veuillez indiquer ici le nombre d'époques final.
k 

**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher ses scores sur les données de test.  Comment se comparent les résultats avec les précédents ?

In [ ]:
# Veuillez écrire votre code ici.


**4c.** Veuillez afficher les scores détaillés pour chaque type de relation de dépendances, dans un tableau formaté comme au 3b.

In [ ]:
# Veuillez écrire votre code ici.


**4d.** Quels changements observez-vous en haut (3 premiers labels) et en bas du classement ?  Voyez-vous un label pour lequel les scores n'augmentent pas avec le parser entraîné ?

**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.